# STAGE B QUIZ.

### NAME: Osaretin Osariemen
### INTERN ID: d97046
### EMAIL ADDRESS: osaretinfrankx@gmail.com

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math
print('Libraries Exported')

Libraries Exported


In [2]:
try:
    df = pd.read_csv('energydata_complete.csv')
except:
    df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
df.head(2)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.7900,19.79,44.73,19.0,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.7225,19.79,44.79,19.0,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195


In [3]:
df.drop(['date','lights'],axis = 1,inplace = True)

In [4]:
scaler = MinMaxScaler()

In [5]:
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
df.head(2)

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,0.046729,0.32735,0.566187,0.225345,0.684038,0.215188,0.746066,0.351351,0.764262,0.175506,...,0.223032,0.677290,0.372990,0.097674,0.894737,0.50000,0.953846,0.538462,0.265449,0.265449
1,0.046729,0.32735,0.541326,0.225345,0.682140,0.215188,0.748871,0.351351,0.782437,0.175506,...,0.226500,0.678532,0.369239,0.100000,0.894737,0.47619,0.894872,0.533937,0.372083,0.372083


## THEORY ONE:   Simple Linear Regression between T2(x-variable) and T6(y-variable).

In [6]:
x1 = df[['T2']]
y1 = df['T6']

In [7]:
lr = LinearRegression()

In [8]:
lr.fit(x1,y1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [9]:
r_sqrd = lr.score(x1,y1).round(2)
print('R-Squared Value is:', r_sqrd)

R-Squared Value is: 0.64


## THEORY TWO: Multiple Linear Regression Model. Appliances as Target(y) variable.

In [10]:
features = df.iloc[:,1:]
target = df.iloc[:,[0]]

In [11]:
x_train,x_test,y_train,y_test = train_test_split(features,target,test_size = 0.3,random_state = 42)

In [12]:
LinearModel = lr.fit(x_train,y_train)

In [13]:
yhat = LinearModel.predict(x_test)

In [14]:
mae = mean_absolute_error(y_test,yhat).round(2)
rmse = np.sqrt(mean_squared_error(y_test,yhat)).round(3)
rss = np.sum(np.square(y_test-yhat)).round(2)
mse = mean_squared_error(y_test,yhat)
r2 = r2_score(y_test,yhat).round(2)

In [15]:
print('''Mean Absolute Error is: {}  \n\nRoot Mean Squared Error is: {}
\nResidual Sum of Squares is: {}  \n\nR-Squared Value is: {}'''.format(mae,rmse,rss,r2))

Mean Absolute Error is: 0.05  

Root Mean Squared Error is: 0.088

Residual Sum of Squares is: Appliances    45.35
dtype: float64  

R-Squared Value is: 0.15


## THEORY THREE: Multiple Linear Model Weights.

In [16]:
#Creating A Custom Weight Generator Function

def model_weights(features,model):
    try:
        dic = {'Features':features.columns,'Weights':model.coef_}
        weights = pd.DataFrame(dic)
        return weights
    except:
        # For the sake of Dimensionality, this turns a 2D array to a 1D array.
        for i in model.coef_:
            model_coef = i
        dic = {'Features':features.columns,'Weights':model_coef}
        weights = pd.DataFrame(dic)
        return weights

In [17]:
# Calling the Function and Assigning it to a Variable.

Linear_weights = model_weights(features,LinearModel)

In [18]:
Linear_weights = Linear_weights.sort_values('Weights',ascending = True).reset_index()

In [19]:
Linear_weights.iloc[[0,-1]]

,index,Features,Weights
0,3,RH_2,-0.456698
25,1,RH_1,0.553547


## THEORY FOUR: Ridge Regression Model.

In [20]:
RR = Ridge(alpha = 0.4)
RidgeModel = RR.fit(x_train,y_train)

In [21]:
yhat2 = RidgeModel.predict(x_test)

In [22]:
rmse = np.sqrt(mean_squared_error(y_test,yhat2)).round(3)
print('Root Mean Squared Error is: ',rmse)

Root Mean Squared Error is:  0.088


#### No Changes or Difference from the RMSE in the Multiple Linear Regression

## THEORY FIVE: Lasso Model.

In [23]:
lasso = Lasso(alpha = 0.001)
LassoModel = lasso.fit(x_train,y_train)

In [24]:
yhat3 = LassoModel.predict(x_test)

In [25]:
# calling previously Created function
Lasso_weights = model_weights(features,LassoModel)

In [26]:
not_zero_weights = Lasso_weights[(Lasso_weights['Weights'] != 0)]
not_zero_weights

,Features,Weights
1,RH_1,0.017880
15,RH_8,-0.000110
20,RH_out,-0.049557
21,Windspeed,0.002912


In [27]:
len(not_zero_weights)

4

#### Features with weight that is not equal to zero are 4 in Numbers.

# END OF NOTEBOOK.